# Sesgo Lingüístico: Comparación Multilingüe (BERT vs GPT-4)

### El Costo del Idioma

Los tokenizadores no son universales; tienen sesgos basados en los datos con los que fueron entrenados (mayormente internet en inglés).

En este experimento, tomamos **la misma frase exacta** traducida a 8 idiomas distintos.
Vamos a comparar:
1.  **Caracteres:** Longitud real del texto.
2.  **Tokens BERT:** Usando `bert-base-uncased` (entrenado solo en inglés).
3.  **Tokens GPT-4:** Usando `cl100k_base` (multilingüe pero centrado en inglés).

**Nota Técnica:** Para BERT, usamos `[1:-1]` al codificar. Esto es para eliminar los tokens automáticos `[CLS]` y `[SEP]` y hacer una comparación justa (manzanas con manzanas) contra GPT-4.

In [1]:
# Instalación de librerías necesarias
try:
    import tiktoken
except ImportError:
    !pip install tiktoken
    import tiktoken

from transformers import BertTokenizer

# Inicializamos los tokenizadores
tokenizerG = tiktoken.get_encoding('cl100k_base') # GPT-4
tokenizerB = BertTokenizer.from_pretrained('bert-base-uncased') # BERT

# Lista de idiomas
languages = ['English','Spanish','Arabic','Persian','Lithuanian','Chinese','Tamil','Esperanto']

# La misma frase traducida (sobre toallas azules y el mar)
sentences = [
    'Blue towels are great because they remind you of the sea, although the sea is wet and towels work better when they are dry.',
    'Las toallas azules son geniales porque recuerdan al mar, aunque el mar está mojado y las toallas funcionan mejor cuando están secas.',
    'تعتبر المناشف الزرقاء رائعة لأنها تذكرك بالبحر، على الرغم من أن البحر مبلل والمناشف تعمل بشكل أفضل عندما تكون جافة.',
    'حوله‌های آبی عالی هستند زیرا شما را به یاد دریا می‌اندازند، اگرچه دریا مرطوب است و حوله‌ها وقتی خشک باشند بهتر عمل می‌کنند.',
    'Mėlyni rankšluosčiai puikūs, nes primena jūrą, nors jūra yra šlapia, o rankšluosčiai geriau tinka, kai yra sausi.',
    '蓝色毛巾很棒，因为它们会让您想起大海，尽管海水是湿的，而毛巾在干燥时效果更好。',
    'நீல நிற துண்டுகள் சிறந்தவை, ஏனென்றால் அவை கடலை நினைவூட்டுகின்றன, இருப்பினும் கடல் ஈரமாக இருக்கும், துண்டுகள் உலர்ந்திருக்கும் போது சிறப்பாக வேலை செய்யும்.',
    'Bluaj mantukoj estas bonegaj ĉar ili memorigas vin pri la maro, kvankam la maro estas malseka kaj mantukoj funkcias pli bone kiam ili estas sekaj.',
]


# Imprimimos la cabecera de la tabla
print(' Language  |  Chars  |  BERT  |  GPT ')
print('-'*37)

for lang, text in zip(languages, sentences):

  # Tokenizamos el texto
  tokensG = tokenizerG.encode(text)

  # Para BERT, cortamos [1:-1] para quitar [CLS] al inicio y [SEP] al final
  tokensB = tokenizerB.encode(text)[1:-1]

  # Imprimimos el resultado alineado
  print(f'{lang:>10} |   {len(text):3}   |  {len(tokensB):3}   |  {len(tokensG):3}')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

 Language  |  Chars  |  BERT  |  GPT 
-------------------------------------
   English |   123   |   26   |   26
   Spanish |   132   |   46   |   34
    Arabic |   115   |   95   |   84
   Persian |   123   |   96   |   90
Lithuanian |   113   |   46   |   57
   Chinese |    39   |   39   |   55
     Tamil |   154   |   35   |  209
 Esperanto |   146   |   56   |   50


### Análisis de los Resultados

Al ejecutar el código, observa las discrepancias:

1.  **La Eficiencia del Inglés:** Verás que English tiene un número bajo de tokens en ambos modelos. Es el "idioma nativo" del tokenizador.
2.  **El Desastre de BERT con otros alfabetos:**
    * Fíjate en **Chinese**, **Arabic** o **Tamil** en la columna BERT. Los números serán altísimos (a veces más tokens que caracteres).
    * **¿Por qué?** Como `bert-base-uncased` no conoce los caracteres árabes o chinos, los trata como "desconocidos" o los rompe byte por byte, inflando el conteo.
3.  **La Mejora de GPT-4:** GPT-4 (`cl100k`) es mucho mejor en idiomas no latinos que el BERT original, pero aun así, idiomas como el Tamil suelen requerir más tokens para expresar la misma idea que el inglés, lo que hace que usar la API sea **más costoso** para usuarios de esos idiomas.